In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(temperature=0.1)

In [9]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text: str):
        items = text.strip().split(",")
        return list(map(str.strip, items))


p = CommaOutputParser()
p.parse("dfd  ,  dfdf")

['dfd', 'dfdf']

In [12]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 리스트 생성 기계입니다. 모든 대답을 콤마 기준으로 하며, 최대 {max_list}개의 리스트로 합니다. 다른 형식의 답변은 하지 않습니다.",
        ),
        ("ai", ""),
        ("human", "{question}"),
    ]
)
prompt = template.format_messages(
    max_list=5, question="대한민국에는 몇개의 도시가 있습니까?"
)
result = llm.predict_messages(prompt)

p.parse(result.content)

['서울', '부산', '대구', '인천', '대전']

In [13]:
# LCEL
chain = template | llm | CommaOutputParser()
chain.invoke({"max_list": 5, "question": "대한민국에는 몇개의 도시가 있습니까?"})

['서울', '부산', '인천', '대구', '대전']